In [2]:
from datasets import load_dataset, Dataset, DatasetDict, load_from_disk
from bigdata_a3_utils import download_all_amazon_reviews, load_compressed_dataset
import polars as pl

In [ ]:
# print(pl.__version__)

In [ ]:
# auto_review: Dataset = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_review_Automotive",
#     split="full",
#     trust_remote_code=True,
# )

# auto_meta: Dataset = load_dataset(
#     "McAuley-Lab/Amazon-Reviews-2023",
#     "raw_meta_Automotive",
#     split="full",
#     trust_remote_code=True,
# )

In [ ]:
# auto_review.to_parquet("amazon_review_auto.parquet")
# auto_meta.to_parquet("amazon_meta_auto.parquet")

In [3]:
def extract_brand() -> pl.Expr:
    store_clean: pl.Then = pl.when(pl.col("store").str.strip_chars() != "").then(pl.col("store"))
    details_brand: pl.Expr = pl.col("details").str.extract(r"Brand[:\s]*([^\s,;]+)", 1)
    return pl.coalesce([store_clean, details_brand, pl.lit("Unknown")]).alias("brand")

In [14]:
lf_review: pl.LazyFrame = pl.scan_parquet("data/raw/review/amazon_review_beauty.parquet")
lf_meta: pl.LazyFrame = pl.scan_parquet("data/raw/meta/amazon_meta_beauty.parquet")

In [15]:
lf_review: pl.LazyFrame = lf_review.filter(pl.col("rating").is_in([1, 2, 3, 4, 5]))
lf_review = lf_review.filter(pl.col("text").str.strip_chars().str.len_chars() > 0)


lf_review = lf_review.with_columns([
    pl.col("text").str.count_matches(r"\b\w+\b").alias("review_length"),
    (pl.col("timestamp").cast(pl.Datetime("ms")).dt.year()).alias("year")
])

In [16]:
lf: pl.LazyFrame = lf_review.join(lf_meta, on="parent_asin", how="left")
lf = lf.with_columns([extract_brand()])
lf = lf.unique(subset=["user_id", "text", "asin"], keep="first")

In [18]:
# df: pl.DataFrame = lf.collect(streaming=True)
print(lf.explain(streaming=True))

STREAMING:
  UNIQUE[maintain_order: false, keep_strategy: First] BY Some(["user_id", "text", "asin"])
     WITH_COLUMNS:
     [when([(col("store").str.strip_chars([null])) != (String())]).then(col("store")).otherwise(null.strict_cast(String)).coalesce([col("details").str.extract([String(Brand[:\s]*([^\s,;]+))]), String(Unknown)]).alias("brand")] 
      LEFT JOIN:
      LEFT PLAN ON: [col("parent_asin")]
         WITH_COLUMNS:
         [col("text").str.count_matches([String(\b\w+\b)]).alias("review_length"), col("timestamp").strict_cast(Datetime(Milliseconds, None)).dt.year().alias("year")] 
          Parquet SCAN [data/raw/review/amazon_review_beauty.parquet]
          PROJECT */10 COLUMNS
          SELECTION: [(col("rating").is_in([Series])) & ([(col("text").str.strip_chars([null]).str.len_chars()) > (0)])]
      RIGHT PLAN ON: [col("parent_asin")]
        Parquet SCAN [data/raw/meta/amazon_meta_beauty.parquet]
        PROJECT */16 COLUMNS
      END LEFT JOIN


In [ ]:
df.write_parquet("data/intermediate/amazon_beauty.parquet")